# Initial EDA

In this notebook, I explore the data for the first time using the 2019 dataset to get a feel for the columns and layout of the data.  

My aim after this exploration is to have a good grasp on what the dataset contains, how things will need to be changed/formatted/engineered in order to create my FSM. 

### Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

parent_dir = '../../'

In [ ]:
from IPython.display import display
pd.options.display.max_columns = None

### Import Data

In [ ]:
prr19 = pd.read_csv(parent_dir + 'data/prr_2019.csv')
prr19.head()

In [ ]:
prr19.info()

## Inital comments

There does not appear to be any documentation for this dataset and so we might need to make some inferences about certain columns and dig deeper into what they mean. For now, I'll detail my current understanding of the column descriptions:

### Column Descriptions

- objectid:  this appears to be the incident ID and should be unique (check this).  This is probably the identifier of the dataset
- zip:  zip code of the incident - currently a float, should probably change to integer
- filenum:  the file number of the incident that is recorded in official police records - check this and make sure it aligns with police records (should be able to look file numbers up...)
- uofnum:  UOF should stand for 'Use of Force'.  The use of force number probably refers to an id number relating to the type of force used.  Should check that uofnum and forcetype match up. 
- occurred_d:  Date the incident occured.  Change to datetime object
- occured_t:  time the incident occured.  Change to datetime object
- current_ba:  Not sure what this column means - will need to look this up.  What does ba mean?  It looks like some sort of identification number - I wonder if it's a number that identifies the police officer?
- offsex:  Sex of the officer 
- offrace:  Race of the officer
- hire_dt:  Date the officer was hired.  Change to datetime object
- off_injure:  Whether the officer was injured during the encounter.
- offcondtype:  What type of condition the officer was in after the incident and/or what injury/ies they had
- off_hospit:  I think this is whether the officer had to go to hospital or not - check this
- service_ty:  I'm not sure what this is exactly but it looks like it is what type of service the officer performed during the incident or what it was labelled as when the officer was called to the scene.  e.g. were they performing an arrest, were they off duty and witnessed a crime, were they attending a service call etc
- forcetype:  This details what type of force was used on the citizen
- street_n:  The street number of the incident
- street:  The street of the incident
- street_g:  This appears to be the direction of the street address (E, S etc)
- street_t:  This is the street type (rd, st, blvd etc)
- address:  The full address (as specified by above fields)
- citnum:  This appears to be some sort of identification number for the citizen.  I'm not clear on this though and I should verify this
- citrace:  Race of the citizen
- citsex:  Sex of the citzen
- cit_injure:  Whether the citizen was injured during the encounter or not
- citcondtyp:  The condition of the citizen after the incident
- cit_arrest:  Whether the citizen was arrested or not (it should be determined whether they were arrested because of the inital incident or arrested because of the resistance (is the dataset on resistance to arrest?  What other types of resistance are there?)
- cit_infl_a:  Was the citizen under the influence of anything during the incident.  'Mentally unstable' is a value in this column so are they counting mental instability as being 'under the influence'?
- citcharget:  Details of what the citizen was being arrest for
- council district:  The ID of the coucil district of the incident
- ra:  Not sure what RA is - look this up
- beat:  The police beat in which the incident occured?  Or is this the police beat from which the officers are from?
- sector:  The police sector?  Or the county section?  Check this
- division:  I think this relates to county info?  But could be more police sector info?  Check this - is it even needed?
- x:  Coordinates of the incident?  Not sure...
- y:  coordinates of the incident?  Not sure...
- geolocation:  Location of the incident
- council districts--test:  Not sure
- dallas city limis gis layer:  Not sure

### Other questions about the data:
- Is the dataset on resistance to arrest?  What other types of resistance are there?
- In the cit_infl_a column, 'Mentally unstable' is a value in this column so are they counting mental instability as being 'under the influence'?

## Data Prep

In [ ]:
# change column names to lower case:
prr19.columns = prr19.columns.str.lower()

In [ ]:
prr19.head()

In [ ]:
prr19.columns

In [ ]:
col_names = ['objectid', 'zip', 'file_num', 'uof_num', 'date_occured', 'time_occured', 'current_ba', 'off_sex', 'off_race', 'hire_date', 'off_injured', 'off_cond_type',
            'off_hospital', 'service_type', 'uof_type', 'uof_reason', 'cycles_num', 'uof_effective', 'street_n', 'street', 'street_g', 'street_t', 'address', 'cit_num', 
            'cit_race', 'cit_sex', 'cit_injured', 'cit_cond_type', 'cit_arrest', 'cit_influence', 'cit_charge_type', 'council_district', 'ra', 'beat', 'sector', 'division',
            'x', 'y', 'geolocation', 'council_districts_test', 'dallas_city_limis_gis_layer']

In [ ]:
len(prr19.columns)

In [ ]:
len(col_names)

In [ ]:
prr19.columns = col_names

In [ ]:
prr19.head()

### Change dtypes and NaNs

Check first for nan's in each column

In [ ]:
prr19.isna().sum()

In [ ]:
prr19[prr19.zip.isna()]

It looks like most of the 'na' values are in relation to the location.  There are 38 entries missing race and 10 missing sex which are more valuable features not to miss. 

Missing the most balues from the cycles_num column which I don't know what this column represents.  I'll have a look at unique values below.  Missing street directions as well but this is to be expected since not all streets have specified directions. 

Other nan's a reasonable since it means it just wasn't relevant to the incident.

In [ ]:
prr19.cycles_num.unique()

In [ ]:
# zip to int:
prr19.zip.unique()

In [ ]:
prr19.zip.isna().sum()

In [ ]:
prr19.astype({'zip': 'int64'}, errors = 'ignore', inplace = True)

Not sure why it won't change zip to an int... I guess it doesn't matter...

In [ ]:
prr19.info()

Change date time objects:

In [ ]:
prr19['date_occured'] = pd.to_datetime(prr19.date_occured)

In [ ]:
prr19['time_occured'] = pd.to_datetime(prr19.time_occured)

In [ ]:
prr19['hire_date'] = pd.to_datetime(prr19.hire_date)

In [ ]:
prr19.info()

### Officer Sex and Race info:

In [ ]:
prr19.off_sex.unique()

In [ ]:
prr19.off_sex.value_counts()

In [ ]:
prr19.off_sex.value_counts(normalize = True)

In [ ]:
# visualise:

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
sns.countplot(x = "off_sex", data = prr19)
ax.set_title('Proportion of Male to Femal Officers', fontsize = 20)
ax.set_xlabel('Sex', fontsize = 18)
ax.set_ylabel('Count', fontsize = 18)
plt.show()

We can see here that over 88% of cops are male.

In [ ]:
prr19.off_race.unique()

In [ ]:
# visualise

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
sns.countplot(x = "off_race", data = prr19)
ax.set_title('Distribution of Officer Races', fontsize = 20)
ax.set_xlabel('Race', fontsize = 18)
ax.set_ylabel('Count', fontsize = 18)
plt.show()

In [ ]:
prr19.off_race.value_counts(normalize = True)

So here we see that 58% of cops are white, 25% are hispanic and 12% are black.

### Citizen Sex and Race info:

In [ ]:
prr19.cit_sex.unique()

In [ ]:
for val in prr19.cit_sex.unique():
    print(type(val))

So we have both 'unknown' and 'nan' values.  It would be best to consolidate these into one group since nan most likely means 'unknown'

In [ ]:
prr19.cit_sex.isna().sum()

In [ ]:
prr19[prr19['cit_sex'] == 'Unknown'].shape

In [ ]:
prr19.cit_sex.replace('Unknown', np.nan, inplace = True)

In [ ]:
prr19.cit_sex.isna().sum()

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
sns.countplot(x = "cit_sex", data = prr19)
ax.set_title('Proportion of Male to Femal Citizens (nan ignored)', fontsize = 20)
ax.set_xlabel('Sex', fontsize = 18)
ax.set_ylabel('Count', fontsize = 18)
plt.show()

In [ ]:
prr19.cit_sex.value_counts(normalize = True)

85% of citizens in this dataset are male.

In [ ]:
prr19.cit_race.unique()

We've got `Unknown`, `nan` and `Other` as categories.  Could these actually be categoriesed into the one category?

In [ ]:
prr19.cit_race.isna().sum()

In [ ]:
prr19[prr19['cit_race'] == 'Unknown'].shape

In [ ]:
prr19[prr19['cit_race'] == 'Other'].shape

So there are 38 'nan', 4 'unknown' and 11 'other' categories.  I think I'm going to combine these for now because I think all signify that a race was not identified. 

In [ ]:
prr19.cit_race.replace('Unknown', np.nan, inplace = True)
prr19.cit_race.replace('Other', np.nan, inplace = True)

In [ ]:
prr19.cit_race.isna().sum()

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
sns.countplot(x = "cit_race", data = prr19)
ax.set_title('Distribution of Citizen Races', fontsize = 20)
ax.set_xlabel('Race', fontsize = 18)
ax.set_ylabel('Count', fontsize = 18)
plt.show()

In [ ]:
prr19.cit_race.value_counts(normalize = True)

So here we see that 55% of citizens in this dataset are black, 22% are white and also 22% are hispanic.  Very small amount are asian and american indian. 

Check where we are:

In [ ]:
prr19.info()

### Count Num of Citizens and Num of Officers

How many officers and how many citizen's in the dataset?  Any repeats?

In [ ]:
# number of unique citizen numbers - at the moment I think these are unique identifiers for individual citizens
len(prr19.cit_num.unique())

So there are only 1649 unique citizen numbers in this dataset which suggests that we have some repeat offenders?

In [ ]:
prr19.cit_num.unique()

In [ ]:
for num in prr19.cit_num.unique():
    print(len(num))

So from looking at the above print outs (it's long), all the cit_nums are multiples of 5, which makes me wonder if it's a combination of citizen numbers for multiple people involved?  I'm not sure what else cit_num could be a I might have to come back to this.

In [ ]:
prr19.cit_num.duplicated().sum()

In [ ]:
prr19[prr19.cit_num.duplicated()].sort_values(by = ['cit_num'])

In [ ]:
cit_num_counts = prr19.cit_num.value_counts()

In [ ]:
cit_num_counts

In [ ]:
cit_num_grt_1 = cit_num_counts[cit_num_counts > 1]

In [ ]:
cit_num_grt_1

Let's investigate the cit_num with the highest count:  78 to get an idea of what it means to have this number duplicated in muliple entries:

In [ ]:
prr19[prr19.cit_num == '6006060061']

So it looks like this cit_num relates to the same citizen and the same event since the `date_occured` column is all on the same day (same with `time_occured`) but there were multiple officers on the scene.  

This gives us a clearer idea of what a row is in this dataset.  It looks like each row is of one police officer's use of force at an incident.  So if there were multiple police officers at the scene that used force, there is one entry for each police officer.  

Let's look at one more of the high count cit_num values:

In [ ]:
prr19[prr19.cit_num == '605076050860509']

This confirms what I've been thinking - that there is a row per officer involved with a particular incident.  **So that means we have 1649 unique citizens in this dataset.**  I'm not sure how would be best to figure out how many unique officers there are in this dataset since there doesn't appear to be a unique identifier for police officers.  We could go by hire date but of course there will be officers hired on the same day.  Let's look into this further.

In [ ]:
# number of unique hire dates:
len(prr19.hire_date.unique())

In [ ]:
prr19.hire_date.value_counts()

In [ ]:
(prr19.hire_date.value_counts() > 1).sum()

So let's have a look at the hire_date entries for the data with 101 rows attached to it to see if this is potentially the same officer or if there are a bunch of officers that were hired on this day:

In [ ]:
prr19[prr19.hire_date == '2018-02-21'].sort_values(by = ['current_ba'])

From this, it looks like `current_ba` is potentially a unique identifier for officers.  We definitely don't have an officer who was involved in 101 incidents - it seems like there are multiple offiers who were hired on February 21, 2018.  With this in mind, let's count the number of unique `current_ba` values to see how many unique offiers are in the dataset.

In [ ]:
len(prr19.current_ba.unique())

So it looks like we have **1015 different officers** in this dataset.  I will continue to verify this as I go along.

### How many unique incidents?

Here, we assume that the `file_num` relates to unique incidents.  Let's investigate and confirm this:

In [ ]:
len(prr19.file_num.unique())

From above, we found that there were potentially 1649 citizens in this dataset, and seeing here that we have 1645 unique file numbers, this suggests that **we have 4 repeat offenders.**

In [ ]:
prr19.file_num.value_counts()

In [ ]:
prr19.cit_num.value_counts()

From here we see that the 3rd-5th cit_nums have bigger counts than the file_num values, and the extras add to exactly 4 so this could indicate that these are our repeat offenders.  Let's check:

In [ ]:
potential_ro = ['37379', '55113', '57295']

In [ ]:
prr19[prr19.cit_num.isin(potential_ro)].sort_values(by = ['cit_num'])

So it appears that for 2 of these citizen numbers, there are different incidents, but for cit_num 55113, there was only 1 incident attached (not a repeat offender).  cit_num 57295 was involved in 2 incidents it appears.  And cit_num 37379 was involved in 10 incidents.  That's 2 repeat offenders which means we need to find the other two.  Come back to this later (and think about what we might need the info of repeat offenders for?  Any model relevance?  Maybe not...)

## Use of Force and Injured Classification

Now investigate use of force types and effectiveness.  We'll also start by visualising the proportion of officers and citizens that are injured during a response to resistance (our target).

### Officer Injured:

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
sns.countplot(x = "off_injured", data = prr19)
ax.set_title('Distribution of Citizen Races', fontsize = 20)
ax.set_xlabel('Race', fontsize = 18)
ax.set_ylabel('Count', fontsize = 18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
sns.countplot(x = "cit_injured", data = prr19)
ax.set_title('Distribution of Citizen Races', fontsize = 20)
ax.set_xlabel('Race', fontsize = 18)
ax.set_ylabel('Count', fontsize = 18)
plt.show()

In [ ]:
off_inj = prr19.off_injured.value_counts()
cit_inj = prr19.cit_injured.value_counts()
injured = pd.DataFrame([off_inj, cit_inj])
injured.index = ['Officers','Citizens']

fig, ax = plt.subplots(figsize = (10, 8))
injured.plot(kind='bar', stacked=True, ax = ax);
plt.suptitle('Percentage of Officers vs. Citizens injured in Response to Resistance', fontsize = 20)
plt.title('Where True (orange) = Injured', fontsize = 18)
ax.set_xlabel('Officers vs. Citizens', fontsize = 18)
ax.set_ylabel('Count', fontsize = 18)
plt.show()

In [ ]:
prr19.off_injured.value_counts(normalize = True)

In [ ]:
prr19.cit_injured.value_counts(normalize = True)

From the above, we can see that just under 10% of officers are injured in response to resistance and 23% of citizen's are injured in response to resistance.  

However, these numbers are misleading given how our data is formatted and what a row represents.  We have to remember than each row is the results of ONE police officer's use of force for the incident they were involved in.  So, if there were 5 officers on the scene and all 5 used force towards the citizen, then perhaps only 1 officer's use of force resulted in injury to the citizen, however, that individual will show 'not injured' for 4 of the other officer's user of force and 'injured' for the 5th, but the overall result is that that citizen was injured during a use of force incident.  So, we need to isolate the data down to incidents, and discover whether the citizen was injured.  Since a row represents 1 officer, the officer injured statistics above should be correct, but we will need to adjust things to account for the citizen being injured.  We could do this using a group by method  (grouping by the file number).  

In [ ]:
prr19.groupby(['file_num']).count()

So, the above dataframe essentially tells us how many cops were involved in a particular incident.  We should then create an aggregate column that lists whether the end result of an incident lead to the injury of the citizen.  

### Use of Force Types:

In [ ]:
len(prr19.uof_type.unique())

So there are 915 different combinations of use of force.  I need to figure out how to seperate these types of force in a way that I can make conclusions about them.

-------
Testing stuff

In [ ]:
ttest = 'hello, yes'
splitted = ttest.split(',')

In [ ]:
splitted

In [ ]:
test = ['hello', 'he, llo', 'what, do, you', 'think, hello', 'what, yes, he']

In [ ]:
unique = []
for item in test:
    split = item.split(',')
    for thing in split:
        thing = thing.strip()
        if thing not in unique:
            unique.append(thing)
unique

-------

In [ ]:
prr19.uof_type.unique()

In [238]:
unique_uof = []
for uof_combo in prr19.uof_type.unique():
    split = uof_combo.split(',')
    for uof in split:
        uof = uof.strip()
        if uof not in unique:
            unique.append(uof)
print(f'Number of unique uses of force: {len(unique_uof)}')
unique_uof

Number of unique uses of force: 0


[]

So in summary, there are 915 combinations of these 32 uses of force. I'm not sure what "BD", "OC" or "LVNR" means...

Will need to think about how I'm going to deal with this column for my model. 

For an FSM I think I might just do 2 things:  
1. Chuck everything and LOL and see what happens
2. Do a reduced version of this where I only put in a subset of the features including demographic information to predict injured/not injured.

## Up next:

Next, I'm going to put together an FSM taking into consideration the findings of this EDA.  Then I'll come back and think about how I'm going to be more thoughtful about how to deal with these features. 

I also need to think about which metric will be best to use as well as being clear on who my audience is here and who this model is for...

Let's see where my current data frame is and export it to a csv for use in my model later. 

In [224]:
prr19.head()

,objectid,zip,file_num,uof_num,date_occured,time_occured,current_ba,off_sex,off_race,hire_date,off_injured,off_cond_type,off_hospital,service_type,uof_type,uof_reason,cycles_num,uof_effective,street_n,street,street_g,street_t,address,cit_num,cit_race,cit_sex,cit_injured,cit_cond_type,cit_arrest,cit_influence,cit_charge_type,council_district,ra,beat,sector,division,x,y,geolocation,council_districts_test,dallas_city_limis_gis_layer
0,2817,75253.0,UF2019-1702,"62295, 63542",2019-12-01,2020-08-04 22:34:00,11285,Male,White,2017-03-08,False,No injuries noted or visible,False,Service Call,"BD - Tripped, BD - Grabbed",Detention/Frisk,NaN,"Yes, Yes",102,Beltline,S,Rd.,102 S Beltline Rd.,60833,White,Male,False,No injuries noted or visible,False,Agitated,No Arrest,D8,6062.0,357.0,350.0,SOUTHEAST,2557123.437,6944231.397,POINT (-96.586265 32.702825),8.0,3.0
1,2234,75208.0,UF2019-1344,61093,2019-10-06,2020-08-04 00:50:00,11208,Male,White,2016-08-24,True,No injuries noted or visible,False,Arrest,Held Suspect Down,Arrest,NaN,Yes,1500,Oak Cliff,S,Blvd.,1500 S Oak Cliff Blvd.,6020748798,Hispanic,Female,True,Injured prior to contact,True,Agitated,APOWW,D1,4160.0,444.0,440.0,SOUTHWEST,2474936.793,6952151.398,POINT (-96.853036 32.729136),1.0,3.0
2,2755,75231.0,UF2019-1665,62820,2019-12-31,2020-08-04 23:37:00,9415,Male,White,2008-04-02,False,No injuries noted or visible,False,Arrest,K-9 Deployment,Arrest,NaN,Yes,6904,Walling,NaN,Ln.,6904 Walling Ln.,61130,Black,Male,True,Bite,True,Poor hygiene,"Burglary/Habitation, Warrant/Hold",D9,6034.0,247.0,240.0,NORTHEAST,2508349.267,7001784.466,POINT (-96.741661 32.863941),13.0,3.0
3,2110,75228.0,UF2019-1314,60990,2019-09-30,2020-08-04 18:20:00,9884,Male,Hispanic,2009-06-10,False,No injuries noted or visible,False,Call for Cover,Joint Locks,Arrest,NaN,Yes,11760,Ferguson,NaN,Rd.,11760 Ferguson Rd.,26625,White,Female,False,No injuries noted or visible,True,Unknown Drugs,"Assault/FV, Resisting Arrest, Warrant/Hold",D9,1132.0,228.0,220.0,NORTHEAST,2536678.324,6999039.025,POINT (-96.649175 32.855492),13.0,3.0
4,1663,75051.0,UF2019-1030,"59592, 59600",2019-08-04,2020-08-04 00:10:00,10480,Male,Hispanic,2012-09-26,True,No injuries noted or visible,False,Arrest,"Joint Locks, BD - Grabbed",Arrest,NaN,"Yes, Yes",1350,Skyline,NaN,Rd.,1350 Skyline Rd.,59513,Black,Male,False,No injuries noted or visible,True,Agitated,Assault/Public Servant,NaN,NaN,NaN,NaN,NaN,2433285.622,6953645.720,POINT (-96.98722 32.734935),NaN,NaN


In [225]:
prr19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 41 columns):
objectid                       2944 non-null int64
zip                            2943 non-null float64
file_num                       2944 non-null object
uof_num                        2944 non-null object
date_occured                   2944 non-null datetime64[ns]
time_occured                   2931 non-null datetime64[ns]
current_ba                     2944 non-null int64
off_sex                        2944 non-null object
off_race                       2944 non-null object
hire_date                      2944 non-null datetime64[ns]
off_injured                    2944 non-null bool
off_cond_type                  2944 non-null object
off_hospital                   2944 non-null bool
service_type                   2937 non-null object
uof_type                       2944 non-null object
uof_reason                     2937 non-null object
cycles_num                     174 non-nul